In [1]:
import pandas as pd
import requests
import warnings
import time
import multiprocessing
import threading
from concurrent.futures import ThreadPoolExecutor
# from google.colab import files
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

In [2]:
# uploaded = files.upload()
# df = pd.read_csv(io.BytesIO(uploaded[list(uploaded.keys())[0]]))
df = pd.read_csv('only_sentence_with_punc.csv')

In [3]:
def subset_worker_words(subset):
  print(subset)
  for row in subset.itertuples():
    print(f"Started {row.Index}")
    if pd.notna(row.ipa):
      print(f"{row.Index}:{row.ipa}")
      continue
    words=row.sentence.split(" ")
    if(len(words)>1):
      datas=""
      for word in words:
        # url= f"http://ipa.bangla.gov.bd/post_text_bangla_ipa?bangla_ipa={word}"
        url= f"http://127.0.0.1:5000//post_text_bangla_ipa?bangla_ipa={word}"
        response=None
        try:
          response = requests.get (url, verify=False)
          if response and response.status_code == 200:
            data= response.json()
            datas=datas+" "+data["data"][0]
          else:
            datas=datas+" "+"ERROR"
        except:
          print(f"error at{row.Index}:{row.sentence}")
          datas=datas+" "+"ERROR"
      print(f"Ended {row.Index}")
      df.at[row.Index,"ipa"]=datas
    else:
      url = f"http://ipa.bangla.gov.bd/post_text_bangla_ipa?bangla_ipa={row.sentence}"
      response=None
      try:
        response = requests.get (url, verify=False)
        if response and response.status_code == 200:
          data= response.json()
          df.at[row.Index,"ipa"]=data["data"][0]
          print(f"Ended {row.Index}:{row.sentence}")
          print(f"{row.Index}:{data['data'][0]}")
        else:
          data= None
      except:
        print(f"error at{row.Index}:{row.sentence}")
  # df.to_csv("withIPA.csv",index=False)
  print("Done")
  return

In [4]:
def subset_worker(subset):
  print(subset)
  for row in subset.itertuples():
    print(f"Started{row.Index}")
    if pd.notna(row.ipa):
      print(f"{row.Index}:{row.ipa}")
      continue
    url = f"http://127.0.0.1:5000//post_text_bangla_ipa?bangla_ipa={row.sentence}"
    response=None
    try:
      response = requests.get (url, verify=False)
      if response and response.status_code == 200:
        data= response.json()
        df.at[row.Index,"ipa"]=data["data"][0]
      else:
        data= None
    except:
      print(f"error at{row.Index}:{row.sentence}")
    print(f"Ended {row.Index}")
    # print(f"{row.Index}:{row.ipa}")
  # df.to_csv("withIPA.csv",index=False)
  print("Done")
  return

In [10]:
def subset_worker_db(subset):
  for row in subset.itertuples():
    data=call_api(row.word)
    if data:
      print(f"{row.Index}:{data['data'][0]}")
    else:
      print(row.Index)
      print(row.word)
      break

In [4]:
# threadpool make sure everything is currect before running this cell
start=0
steps=100
end=1000
subsets=[]
start_pos=start
while start<end:
  subsets.append(df.iloc[start:start+steps])
  start=start+steps
print(subsets)
pool=ThreadPoolExecutor(len(subsets))
ws=[]
for subset in subsets:
  ws.append(pool.submit(subset_worker_words, subset))
print(pool.shutdown())
df.to_csv(f"only_sentence_with_punc_ipa{start_pos}_{end}.csv",index=False)

[    e_index                                           sentence  ipa
0         0  ৯নব্যভারত (ষোড়শ খণ্ড, বৈশাখ-চৈত্র, ১৩০৫), ২৮০...  NaN
1         0  তবে ১৮৭২ খ্রিস্টাব্দে বঙ্গদর্শন-এর প্রকাশ এই ধ...  NaN
2         0  বস্তুত বাগ্মিতায় অক্ষয়কুমারের ছিল জন্মগত অধি...  NaN
3         0  ভারতীয় শিল্পের কথা নামক পুস্তকে ইতিপূর্বে শিল...  NaN
4         0  অন্যান্য বিষয়ের সঙ্গে এ পত্রিকায় ঐতিহাসিক প্...  NaN
..      ...                                                ...  ...
95        8     কিন্তু সেফালিকার ফুল অপেক্ষা বৃন্তের গুণ অধিক।  NaN
96        8  কাঁঠাল কাষ্ঠের সারভাবা জলে ভিজাইয়া কাথ বাহির ...  NaN
97        8  ভারতবর্ষ ভিন্ন ইংরাজাধিকৃত রাজ্যের অন্য কোনো স...  NaN
98        8  বৃক্ষোৎপন্ন লাক্ষা গুঁড়া-করিয়া জল ঢালিয়া তা...  NaN
99        8  কৰ্তব্য শেষ করিয়া আত্মপ্রসাদ লাভ করিবার উপায়...  NaN

[100 rows x 3 columns],      e_index                                           sentence  ipa
100        8  রং বাহির করিয়া লইলে যে শিটি বা ছিবড়ে পরিত্যক...  NaN
101        8  বিশুদ্

In [ ]:
# Fills the empty ones
subset1_1000=df.iloc[3000:6000]
for row in subset1_1000.itertuples():
  print(row.Index)
  # print(row[1])
  if pd.notna(row.ipa):
    continue
  print(row.sentence)
  data=call_api(row.sentence)
  # print(data["data"][0])
  if data:
    df.at[row.Index,"ipa"]=data["data"][0]
  else:
    print(row.Index)
    print(row.sentence)
    break
df.to_csv("withIPA.csv",index=False)

**TESTINS**

---



In [3]:
# test connection
def call_api (param):
    url = f"http://127.0.0.1:5000//transcribeBanglaTextToIpa?bangla_ipa={param}"
    response = requests.get (url, verify=False)
    if response.status_code == 200:
        return response.json()
    else:
        return None
data=call_api("আমার সোনার বাংলা।")
data["data"][0]

'amar ʃonar baŋla।'

In [ ]:
# Fills the empty ones
subset1_1000=df.iloc[20000:70000]
for row in subset1_1000.itertuples():
  # print(row.Index)
  # print(row[1])
  # if pd.notna(row.ipa):
  #   continue
  # print(row.word)
  data=call_api(row.word)
  # print(data["data"][0])
  if data:
    # df.at[row.Index,"ipa"]=data["data"][0]
    print(f"{row.Index}:{data['data'][0]}")
  else:
    print(row.Index)
    print(row.word)
    break
# df.to_csv("withIPA.csv",index=False)

In [ ]:
multiprocessing.cpu_count()

In [43]:
print(pool.shutdown( cancel_futures=True))
df.to_csv(f"withIPA{start_pos}_{end}.csv",index=False)

None


In [ ]:

# w0= pool.submit(subset_worker, subsets[0])
# w1=pool.submit(subset_worker, subsets[1])
# w2=pool.submit(subset_worker, subsets[2])
# w3=pool.submit(subset_worker, subsets[3])
# w4=pool.submit(subset_worker, subsets[4])
# w0.result()
# w1.result()
# w2.result()
# w3.result()
# w4.result()

# print(w0.cancel())
# print(w1.cancel())
# print(w2.cancel())
# print(w3.cancel())
# print(w4.cancel())


In [ ]:
# not working
i=500
steps=100
end=1000
subsets=[]
while i<end:
  subsets.append(df.iloc[i:i+steps])
  i=i+steps
print(subsets)

threads=[]
for subset in subsets:
  threads.append(threading.Thread(target=subset_worker, daemon=True, args=(subset,)))
for thread in threads:
  thread.start();
  time.sleep(1)
for thread in threads:
  thread.join()
print("end")

Cleaning

In [97]:
# Unique word clearing
df=pd.read_csv("bangla_to_ipa_unique_clean2.csv")
df.drop_duplicates(subset=['bangla_token'], inplace=True)
print(len(df))
df.sort_values('bangla_token').to_csv("bangla_to_ipa_unique_clean.csv",index=False)



58569


In [99]:
df=pd.read_csv("bangla_to_ipa_unique_clean2.csv")
remove=[]
for row in df.itertuples():
  if len(row.bangla_token)>2 and row.bangla_token[-2]=="!":
    # print(row.bangla_token[1:])

    for i in range(1,110,1):
      if df.at[row.Index-i,"bangla_token"]==row.bangla_token[:-2]:
        print(row.Index)
        print(row.bangla_token)
        # remove that row
        remove.append(row.Index)
        break

df.drop(remove, inplace=True)
print(len(remove))

0


In [113]:
# check whole dataset
df=pd.read_csv("bangla_to_ipa_unique_clean2.csv")
remove=[]
for row in df.itertuples():
  if row.bangla_token[:-1]=="?":
    # if contains in the whole dataset
    for i in range(len(df)):
      if df.at[i,"bangla_token"]==row.bangla_token[:-1]:
        print(row.Index)
        print(i)
        print(row.bangla_token)
        # remove that row
        remove.append(row.Index)
        break
df.drop(remove, inplace=True)


In [110]:
df.to_csv("bangla_to_ipa_unique_clean2.csv",index=False)

In [83]:
a="bangla_to_ipa_unique_clean"
print(a[1:])

angla_to_ipa_unique_clean
